In [1]:
import whisper
import ffmpeg
import numpy as np
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
#whisper_model = whisper.load_model("base").to(device)

# Get input audio

In [3]:
import pyaudio
import wave

def record_clip(DEVICE_INDEX = 10, FORMAT = pyaudio.paInt16, RATE=48000, CHUNK=1024, RECORD_SECONDS=6,
                WAVE_OUTPUT_FILENAME = 'filename.wav'):

    audio = pyaudio.PyAudio()
    
    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK,
                        input_device_index=DEVICE_INDEX)
                        # sample_rate=RATE)
    print("recording...")
    print('---------------------------------')
    print(int(RATE / CHUNK * RECORD_SECONDS))
    print('*********************************')

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        print("Recording . . .")

        frames.append(data)
    print("Recording finished. . .")

    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(2)
    audio.get_sample_size(FORMAT)
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

# Process audio and turn to TEXT

# TEXT to SPEECH

In [9]:
from TTS.api import TTS

class TextToSpeech:
    def __init__(self):
        

        # Running a multi-speaker and multi-lingual model

        # List available 
        model_name = TTS.list_models()[0]
        # Init TTS
        self.tts = TTS(model_name, progress_bar = False, gpu = True)
    def text_to_speech():
        0
    def text_to_speech_to_file(self, text, output = 'output.wav'):
        # Run TTS
        self.tts.tts_to_file(text=text, speaker=self.tts.speakers[3], language=self.tts.languages[0], 
                             file_path="output.wav")
        #wav = tts.tts("This is a test! This is also a test!!", speaker=tts.speakers[1], language=tts.languages[0])
        # Text to speech to a file
        #tts.tts_to_file(text="Hello world!", speaker=tts.speakers[0], language=tts.languages[0], file_path="output.wav")
txt2spch = TextToSpeech()

 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.


ValueError:  [!] Model file not found in the output path

# ChatGPT API

In [ ]:
# TODO - Get a ChatGPT API which actually works.

# Streaming audio

In [1]:
import time, logging
from datetime import datetime
import threading, collections, queue, os, os.path
import numpy as np
import pyaudio
import wave
import webrtcvad
from halo import Halo
from scipy import signal

logging.basicConfig(level=20)

class Audio(object):
    """Streams raw audio from microphone. Data is received in a separate thread, and stored in a buffer, to be read from."""

    FORMAT = pyaudio.paInt16
    # Network/VAD rate-space
    RATE_PROCESS = 16000
    CHANNELS = 1
    BLOCKS_PER_SECOND = 50

    def __init__(self, callback=None, device=None, input_rate=RATE_PROCESS, file=None):
        def proxy_callback(in_data, frame_count, time_info, status):
            #pylint: disable=unused-argument
            if self.chunk is not None:
                in_data = self.wf.readframes(self.chunk)
            callback(in_data)
            return (None, pyaudio.paContinue)
        if callback is None: callback = lambda in_data: self.buffer_queue.put(in_data)
        self.buffer_queue = queue.Queue()
        self.device = device
        self.input_rate = input_rate
        self.sample_rate = self.RATE_PROCESS
        self.block_size = int(self.RATE_PROCESS / float(self.BLOCKS_PER_SECOND))
        self.block_size_input = int(self.input_rate / float(self.BLOCKS_PER_SECOND))
        self.pa = pyaudio.PyAudio()

        kwargs = {
            'format': self.FORMAT,
            'channels': self.CHANNELS,
            'rate': self.input_rate,
            'input': True,
            'frames_per_buffer': self.block_size_input,
            'stream_callback': proxy_callback,
        }

        self.chunk = None
        # if not default device
        if self.device:
            kwargs['input_device_index'] = self.device
        elif file is not None:
            self.chunk = 320
            self.wf = wave.open(file, 'rb')
        print("KWARGS:", kwargs)
        self.stream = self.pa.open(**kwargs)
        self.stream.start_stream()

    def resample(self, data, input_rate):
        """
        Microphone may not support our native processing sampling rate, so
        resample from input_rate to RATE_PROCESS here for webrtcvad and
        deepspeech
        Args:
            data (binary): Input audio stream
            input_rate (int): Input audio rate to resample from
        """
        data16 = np.fromstring(string=data, dtype=np.int16)
        resample_size = int(len(data16) / self.input_rate * self.RATE_PROCESS)
        resample = signal.resample(data16, resample_size)
        resample16 = np.array(resample, dtype=np.int16)
        return resample16.tostring()

    def read_resampled(self):
        """Return a block of audio data resampled to 16000hz, blocking if necessary."""
        return self.resample(data=self.buffer_queue.get(),
                             input_rate=self.input_rate)

    def read(self):
        """Return a block of audio data, blocking if necessary."""
        return self.buffer_queue.get()

    def destroy(self):
        self.stream.stop_stream()
        self.stream.close()
        self.pa.terminate()

    frame_duration_ms = property(lambda self: 1000 * self.block_size // self.sample_rate)

    def write_wav(self, filename, data):
        logging.info("write wav %s", filename)
        wf = wave.open(filename, 'wb')
        wf.setnchannels(self.CHANNELS)
        # wf.setsampwidth(self.pa.get_sample_size(FORMAT))
        assert self.FORMAT == pyaudio.paInt16
        wf.setsampwidth(2)
        wf.setframerate(self.sample_rate)
        wf.writeframes(data)
        wf.close()


class VADAudio(Audio):
    """Filter & segment audio with voice activity detection."""

    def __init__(self, aggressiveness=3, device=None, input_rate=None, file=None):
        super().__init__(device=device, input_rate=input_rate, file=file)
        self.vad = webrtcvad.Vad(aggressiveness)

    def frame_generator(self):
        """Generator that yields all audio frames from microphone."""
        if self.input_rate == self.RATE_PROCESS:
            while True:
                yield self.read()
        else:
            while True:
                yield self.read_resampled()

    def vad_collector(self, padding_ms=300, ratio=0.75, frames=None):
        """Generator that yields series of consecutive audio frames comprising each utterence, separated by yielding a single None.
            Determines voice activity by ratio of frames in padding_ms. Uses a buffer to include padding_ms prior to being triggered.
            Example: (frame, ..., frame, None, frame, ..., frame, None, ...)
                      |---utterence---|        |---utterence---|
        """
        if frames is None: frames = self.frame_generator()
        num_padding_frames = padding_ms // self.frame_duration_ms
        ring_buffer = collections.deque(maxlen=num_padding_frames)
        triggered = False

        for frame in frames:
            if len(frame) < 640:
                return

            is_speech = self.vad.is_speech(frame, self.sample_rate)

            if not triggered:
                ring_buffer.append((frame, is_speech))
                num_voiced = len([f for f, speech in ring_buffer if speech])
                if num_voiced > ratio * ring_buffer.maxlen:
                    triggered = True
                    for f, s in ring_buffer:
                        yield f
                    ring_buffer.clear()

            else:
                yield frame
                ring_buffer.append((frame, is_speech))
                num_unvoiced = len([f for f, speech in ring_buffer if not speech])
                if num_unvoiced > ratio * ring_buffer.maxlen:
                    triggered = False
                    yield None
                    ring_buffer.clear()

def main(nospinner = True, model=None, savewav = False, vad_aggressiveness=3, device = 7, rate = 48000,
        file = 'out.wav'):
    
    # Start audio with VAD
    vad_audio = VADAudio(aggressiveness=vad_aggressiveness,
                         device=device,
                         input_rate=rate,
                         file=file)
    print("Listening (ctrl-C to exit)...")
    frames = vad_audio.vad_collector()

    # Stream from microphone to the model using VAD
    spinner = None
    if not nospinner:
        spinner = Halo(spinner='line')
    #stream_context = model.createStream()
    wav_data = bytearray()
    for frame in frames:
        if frame is not None:
            if spinner: spinner.start()
            logging.debug("streaming frame")
            #stream_context.feedAudioContent(np.frombuffer(frame, np.int16))
            if savewav: wav_data.extend(frame)
        else:
            if spinner: spinner.stop()
            logging.debug("end utterance")
            if savewav:
                vad_audio.write_wav(os.path.join(savewav, datetime.now().strftime("savewav_%Y-%m-%d_%H-%M-%S_%f.wav")), wav_data)
                wav_data = bytearray()
            #text = stream_context.finishStream()
            #print("Recognized: %s" % text)
            #stream_context = model.createStream()

In [2]:
def pyaudio_get_input_mic_device(searching_for = "Logitech"):
    # Input params
    # Searching for - partial name of device which should be used. UPPER AND LOWER CASE MATTERS ! 

    # Find device
    good_device_info = None
    aud = pyaudio.PyAudio()
    for i in range(0,15):
        try:
            info = aud.get_device_info_by_index(i)
            #print(info['name'])
            if searching_for in info['name']:
                print(info)
                good_device_info = info
        except Exception as e :
            print("Exception:", e)
            
    aud.terminate()
    if good_device_info is None:
        raise Exception("Did not find a good input device !")
    else:
        return good_device_info

In [3]:
# Get good device info
device_info = pyaudio_get_input_mic_device()
# Initialize VAD and stuffs
main(nospinner = True, model=None, savewav = False, vad_aggressiveness=3, device = device_info['index'], rate = int(device_info['defaultSampleRate']),
        file = 'out.wav')

{'index': 10, 'structVersion': 2, 'name': 'Logitech USB Headset: Audio (hw:2,0)', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.008, 'defaultLowOutputLatency': 0.008, 'defaultHighInputLatency': 0.032, 'defaultHighOutputLatency': 0.032, 'defaultSampleRate': 48000.0}
KWARGS: {'format': 8, 'channels': 1, 'rate': 48000, 'input': True, 'frames_per_buffer': 960, 'stream_callback': <function Audio.__init__.<locals>.proxy_callback at 0x7fcfa0193310>, 'input_device_index': 10}
Listening (ctrl-C to exit)...


ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
/tmp/ipykernel_379/1496248536.py:67: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data16 = np.fromstring(string=data, dtype=np.int16)
/tmp/ipykernel_379/1496248536.py:71: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return resample16.tostring()


KeyboardInterrupt: 

In [20]:
kw = {'format': 8, 'channels': 1, 'rate': 48000, 'input': True, 'frames_per_buffer': 960, 'stream_callback': None,
      'input_device_index': 10}
aud = pyaudio.PyAudio()

In [22]:
stream = aud.open(input_device_index = kw['input_device_index'], rate = kw['rate'], channels = kw['channels'],
                 format = kw['format'])

ValueError: Must specify an input or output stream.

### Whisper testing

In [17]:
def whisper_to_text(whisper_model = None, audiofile = None, steamed_frame = None):
    if audiofile is not None:
    # load audio and pad/trim it to fit 30 seconds
        audio = whisper.load_audio(audiofile)
        
    else:
        audio = steamed_frame
        0
    audio = whisper.pad_or_trim(audio)
    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)

    # detect the spoken language
    _, probs = whisper_model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions(fp16 = False, language = 'english')
    result = whisper.decode(whisper_model, mel, options)

    # print the recognized text
    print(result.text)
    output = result.text
    return output

whisper_to_text(audiofile =None, whisper_model = whisper_model, steamed_frame = out)

Detected language: en
I'm so much for you.


"I'm so much for you."

In [16]:
sr = 48000

try:
    # This launches a subprocess to decode audio while down-mixing and resampling as necessary.
    # Requires the ffmpeg CLI and `ffmpeg-python` package to be installed.
    out, _ = (
        ffmpeg.input('filename', threads=0)
        .output("-", format="s16le", acodec="pcm_s16le", ac=1, ar=sr)
        .run(cmd=["ffmpeg", "-nostdin"], capture_stdout=True, capture_stderr=True)
    )
except ffmpeg.Error as e:
    raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

out = np.frombuffer(out, np.int16).flatten().astype(np.float32) / 32768.0

In [11]:
out.shape

(287744,)

In [ ]:
def streamed_frame_to_whisper():
    